# Ideal camera placement to world view

In [ ]:
# Link to good article: https://ksimek.github.io/2012/08/22/extrinsic/

import numpy as np
import matplotlib.pyplot as plt

#%matplotlib widget

world_origin = [0, 0, 0]    # Set world origin coordinate to [0, 0, 0]

# Generate 3d coordinates for cameras
thetas = np.arange(0,360,5)
phis = np.arange(45,135+5,5)

print(f'theta: {thetas}\nphi: {phis}')

camera_pose = []
xl = []
yl = []
zl = []
for phi in phis:
    for theta in thetas:
        x = np.cos(theta*np.pi/180.) * np.sin(phi*np.pi/180.)
        y = np.sin(theta*np.pi/180.) * np.sin(phi*np.pi/180.)
        z = np.cos(phi*np.pi/180.)
        xl.append(x)
        yl.append(y)
        zl.append(z)
        camera_pose.append([x, y, z])
        #distance = np.sqrt(x**2 + y**2 + z**2)
        #print(f'phi: {phi}, theta: {theta}, distance: {distance:.2f}, camera pose: {camera_pose[-1]}')
camera_pose = np.array(camera_pose)

print(f'Total cameras: {len(phis)*len(thetas)}')
fig = plt.figure(figsize=(12, 12))
ax = plt.axes(projection='3d')
ax.scatter(camera_pose[:,0], camera_pose[:,1], camera_pose[:,2])    # ax.scatter(xl, yl, zl)
for pose in camera_pose:
        ax.plot([world_origin[0], pose[0]], [world_origin[1], pose[1]], [world_origin[2], pose[2]])

## Display camera to world perspective

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

#%matplotlib widget

world_origin = [0, 0, 0]    # Set world origin coordinate to [0, 0, 0]

file_path = 'C:/_SW/eb_python/cv/fox/train/pose/'

pose_file_names = [f for f in os.listdir(file_path) if f.endswith('.txt')]
#print(pose_file_names)
cams_extrinsic = []
for file_name in pose_file_names:
    #print(f'{file_path}{file_name}')
    s_extrinsic = open(f'{file_path}{file_name}').read().split()
    extrinsic = np.array(s_extrinsic, dtype=float).reshape(4, 4)
    cams_extrinsic.append(extrinsic)
    #distance = np.sqrt(extrinsic[0,3]**2 + extrinsic[1,3]**2 + extrinsic[2,3]**2)
    #print(f'Distance: {distance:.2f}')
cams_extrinsic = np.array(cams_extrinsic)
cams_position = cams_extrinsic[:, :3, 3]
cams_rotation = cams_extrinsic[:, :3, 0:3]


cam_direction = np.array([0, 0, 90])
cams_direction = []
for cam_rotation in cams_rotation:
    #cams_direction.append(cam_rotation.dot(cam_direction))
    world_direction = cam_rotation.dot(cam_direction)
    world_direction /= np.linalg.norm(world_direction)
    cams_direction.append(world_direction)
cams_direction = np.array(cams_direction)
#print(cams_direction)

scatter_trace = go.Scatter3d(
    x=cams_position[:, 0],
    y=cams_position[:, 1],
    z=cams_position[:, 2],
    mode='markers',
    marker=dict(
        size=5,
        opacity=0.7
    )
)

line_traces = []
for i in range(cams_position.shape[0]):
    line_trace = go.Scatter3d(
        x=[cams_position[i, 0], cams_direction[i][0]],
        y=[cams_position[i, 1], cams_direction[i][1]],
        z=[cams_position[i, 2], cams_direction[i][2]],
        mode='lines',
        line=dict(
            color='blue',  # Specify the color of the line
            width=2  # Specify the width of the line
        )
    )
    line_traces.append(line_trace)

fig = go.Figure(data=[scatter_trace, *line_traces])
fig.update_layout(width=1200, height=1200)
fig.show()

# Display XYZ axes for cameras and world

In [1]:
def import_intrinsics(file_path):
    file_names = [f for f in os.listdir(file_path) if f.endswith('.txt')]
    cams_intrinsic = []
    for file_name in file_names:
        s_intrinsic = open(f'{file_path}{file_name}').read().split()
        intrinsic = np.array(s_intrinsic, dtype=float).reshape(4, 4)
        cams_intrinsic.append(intrinsic)
    
    return np.array(cams_intrinsic)

In [2]:
def import_extrinsics(file_path):
    file_names = [f for f in os.listdir(file_path) if f.endswith('.txt')]
    cams_extrinsic = []
    for file_name in file_names:
        s_extrinsic = open(f'{file_path}{file_name}').read().split()
        extrinsic = np.array(s_extrinsic, dtype=float).reshape(4, 4)
        cams_extrinsic.append(extrinsic)
    
    return np.array(cams_extrinsic)

In [3]:
def cam2world(camera_point, camera_extrinsic):
    rotation_matrix = camera_extrinsic[:3, :3]
    translation_vector = camera_extrinsic[:3, 3]
    
    rotated_camera_point = rotation_matrix @ camera_point
    world_point = rotated_camera_point + translation_vector

    return world_point

In [27]:
def pixel2world(x, y, camera_intrinsic, camera_extrinsic, zoom_factor=1):
    fx, fy, cx, cy = camera_intrinsic[0, 0], camera_intrinsic[1, 1], camera_intrinsic[0, 2], camera_intrinsic[1, 2]
    #print(f'fx: {fx}, fy:{fy}, cx: {cx}, cy: {cy}')
    
    # Normalize pixel coordinates
    u_prime = (x - cx) / fx
    v_prime = (y - cy) / fy
    #print(f'u: {u_prime}, v:{v_prime}')
    
    homogeneous_coordinates = np.array([u_prime, v_prime, 1])
    #print(f'Homogeneous coordinates: {homogeneous_coordinates}')
    euclidian_distance = np.sqrt(homogeneous_coordinates[0]**2 + homogeneous_coordinates[1]**2 + homogeneous_coordinates[2]**2)
    #print(f'Euclidian distance: {euclidian_distance}')

    # Convert to camera coordinates using the inverse of the intrinsic matrix
    #camera_coordinates = np.linalg.inv(camera_intrinsic[:3, :3]).dot(homogeneous_coordinates)
    #print(f'Camera coordinates: {camera_coordinates}')

    camera_coordinates = homogeneous_coordinates / euclidian_distance * zoom_factor
    world_coordinates = cam2world(camera_coordinates, camera_extrinsic)
    #print(f'World coordinates: {world_coordinates}')
    
    return world_coordinates

In [6]:
def gen_axes(cameras_extrinsic):
    axes = []

    axes_length = 1.
    axis_o, axis_x, axis_y, axis_z = [0, 0, 0], [axes_length, 0, 0], [0, axes_length, 0], [0, 0, axes_length]
    axes.append([np.concatenate([axis_o, axis_x], axis=0),   # X axis
                 np.concatenate([axis_o, axis_y], axis=0),   # Y axis
                 np.concatenate([axis_o, axis_z], axis=0)])  # Z axis
    
    axes_length = 0.4
    axis_o, axis_x, axis_y, axis_z = [0, 0, 0], [axes_length, 0, 0], [0, axes_length, 0], [0, 0, axes_length]
    for camera_extrinsic in cameras_extrinsic:
        world_o = cam2world(axis_o, camera_extrinsic)
        world_x = cam2world(axis_x, camera_extrinsic)
        world_y = cam2world(axis_y, camera_extrinsic)
        world_z = cam2world(axis_z, camera_extrinsic)
        axes.append([np.concatenate([world_o, world_x], axis=0),   # X axis
                     np.concatenate([world_o, world_y], axis=0),   # Y axis
                     np.concatenate([world_o, world_z], axis=0)])  # Z axis
    
    return np.array(axes)

In [28]:
def gen_img_frame(cameras_intrinsic, cameras_extrinsic):
    frames = []
    
    for camera_intrinsic, camera_extrinsic in zip(cameras_intrinsic, cameras_extrinsic):
        world_pt1 = pixel2world(0, 0, camera_intrinsic, camera_extrinsic, zoom_factor=10.)
        world_pt2 = pixel2world(400, 0, camera_intrinsic, camera_extrinsic, zoom_factor=10.)
        world_pt3 = pixel2world(400, 400, camera_intrinsic, camera_extrinsic, zoom_factor=10.)
        world_pt4 = pixel2world(0, 400, camera_intrinsic, camera_extrinsic, zoom_factor=10.)
        frames.append([np.concatenate([world_pt1, world_pt2], axis=0),
                       np.concatenate([world_pt2, world_pt3], axis=0),
                       np.concatenate([world_pt3, world_pt4], axis=0),
                       np.concatenate([world_pt4, world_pt1], axis=0)])
    
    return np.array(frames)

In [31]:
def add_point(renderer, position, color=(1, 0, 0), size=10):
    points = vtk.vtkPoints()
    points.InsertNextPoint(position)

    polyData = vtk.vtkPolyData()
    polyData.SetPoints(points)

    glyphFilter = vtk.vtkVertexGlyphFilter()
    glyphFilter.SetInputData(polyData)
    glyphFilter.Update()

    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(glyphFilter.GetOutputPort())

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetPointSize(size)
    actor.GetProperty().SetColor(color)

    renderer.AddActor(actor)


def add_line(renderer, start_point, end_point, color=(0, 1, 0), width=2):
    points = vtk.vtkPoints()
    points.InsertNextPoint(start_point)
    points.InsertNextPoint(end_point)

    line = vtk.vtkLine()
    line.GetPointIds().SetId(0, 0)  # the index of the start point
    line.GetPointIds().SetId(1, 1)  # the index of the end point

    lines = vtk.vtkCellArray()
    lines.InsertNextCell(line)

    linePolyData = vtk.vtkPolyData()
    linePolyData.SetPoints(points)
    linePolyData.SetLines(lines)

    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(linePolyData)

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(color)
    actor.GetProperty().SetLineWidth(width)

    renderer.AddActor(actor)


def draw_solid_rectangle(renderer, points_list, color=(0.5, 0.5, 0.5)):
    # Ensure there are exactly four points
    if len(points_list) != 4:
        raise ValueError("points_list must contain exactly four points.")

    # Create the points and the polygon
    points = vtk.vtkPoints()
    polygon = vtk.vtkPolygon()
    polygon.GetPointIds().SetNumberOfIds(4)  # Rectangle has 4 vertices

    for i, (x, y, z) in enumerate(points_list):
        points.InsertNextPoint(x, y, z)
        polygon.GetPointIds().SetId(i, i)

    polygons = vtk.vtkCellArray()
    polygons.InsertNextCell(polygon)

    # Create a PolyData object to hold the polygon data
    polyData = vtk.vtkPolyData()
    polyData.SetPoints(points)
    polyData.SetPolys(polygons)

    # Create a mapper and actor for the polygon
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(polyData)

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(color)  # Set the color of the rectangle

    # Add the actor to the renderer
    renderer.AddActor(actor)


def add_png_img(renderer, image_path, rectangle_coords):
    # Ensure rectangle_coords has four sets of (x, y, z) coordinates
    if len(rectangle_coords) != 4:
        raise ValueError("rectangle_coords must contain exactly four (x, y, z) coordinate tuples.")

    # Create the points and polygon for the rectangle
    points = vtk.vtkPoints()
    polygon = vtk.vtkPolygon()
    polygon.GetPointIds().SetNumberOfIds(4)  # A rectangle has four corners

    for i, (x, y, z) in enumerate(rectangle_coords):
        points.InsertNextPoint(x, y, z)
        polygon.GetPointIds().SetId(i, i)

    polygons = vtk.vtkCellArray()
    polygons.InsertNextCell(polygon)

    # Create a PolyData
    polyData = vtk.vtkPolyData()
    polyData.SetPoints(points)
    polyData.SetPolys(polygons)

    # Texture coordinates
    textureCoordinates = vtk.vtkFloatArray()
    textureCoordinates.SetNumberOfComponents(2)  # (u, v) pairs
    textureCoordinates.SetName("TextureCoordinates")
    textureCoordinates.InsertNextTuple((0.0, 0.0))
    textureCoordinates.InsertNextTuple((1.0, 0.0))
    textureCoordinates.InsertNextTuple((1.0, 1.0))
    textureCoordinates.InsertNextTuple((0.0, 1.0))
    polyData.GetPointData().SetTCoords(textureCoordinates)

    # Apply the texture
    reader = vtk.vtkPNGReader()
    reader.SetFileName(image_path)
    texture = vtk.vtkTexture()
    texture.SetInputConnection(reader.GetOutputPort())

    # Map texture to polydata
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(polyData)

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.SetTexture(texture)

    renderer.AddActor(actor)

In [39]:
import vtk
import os
import numpy as np

intrinsics_path = 'C:/_SW/eb_python/cv/fox/train/intrinsics/'
extrinsics_path = 'C:/_SW/eb_python/cv/fox/train/pose/'
images_path = 'C:/_SW/eb_python/cv/fox/imgs/'

cameras_intrinsic = import_intrinsics(intrinsics_path)
cameras_extrinsic = import_extrinsics(extrinsics_path)
axes = gen_axes(cameras_extrinsic)
frames = gen_img_frame(cameras_intrinsic, cameras_extrinsic)

# Create a renderer, render window, and interactor
renderWindow = vtk.vtkRenderWindow()
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
style = vtk.vtkInteractorStyleTrackballCamera()
renderWindowInteractor.SetInteractorStyle(style)
renderer = vtk.vtkRenderer()

# Enable depth peeling
renderWindow.SetAlphaBitPlanes(1)  # Use alpha bit-planes
renderWindow.SetMultiSamples(0)  # Disable multi-sampling for depth peeling

renderer.SetUseDepthPeeling(1)
renderer.SetMaximumNumberOfPeels(100)
renderer.SetOcclusionRatio(0.1)

renderWindow.AddRenderer(renderer)

for axis in range(axes.shape[0]):
    add_line(renderer, axes[axis,0,:3], axes[axis,0,3:], color=(1,0,0))
    add_line(renderer, axes[axis,1,:3], axes[axis,1,3:], color=(0,1,0))
    add_line(renderer, axes[axis,2,:3], axes[axis,2,3:], color=(0,0,1))

for frame in range(frames.shape[0]):
    add_line(renderer, frames[frame,0,:3], frames[frame,0,3:], color=(1,0,0))
    add_line(renderer, frames[frame,1,:3], frames[frame,1,3:], color=(0,0,0))
    add_line(renderer, frames[frame,2,:3], frames[frame,2,3:], color=(0,0,0))
    add_line(renderer, frames[frame,3,:3], frames[frame,3,3:], color=(0,1,0))
    rectangle_coords = [tuple(frames[frame,0,:3]), tuple(frames[frame,1,:3]), tuple(frames[frame,2,:3]), tuple(frames[frame,3,:3])]
    draw_solid_rectangle(renderer, rectangle_coords, color=(1, 1, 1))

img_file_names = [f for f in os.listdir(images_path ) if f.endswith('.png')]
for i, img_file_name in enumerate(img_file_names):
    rectangle_coords = [tuple(frames[i,0,:3]), tuple(frames[i,1,:3]), tuple(frames[i,2,:3]), tuple(frames[i,3,:3])]
    add_png_img(renderer, f'{images_path}{img_file_name}', rectangle_coords)

# Render and start interaction
renderer.SetBackground(255, 255, 255)  # Background color
renderWindow.Render()
renderWindowInteractor.Start()